In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from numpy import random
import matplotlib.pyplot as plt
import seaborn as sns
from torch.utils.data import DataLoader
from collections import deque
import numpy as np
import multiprocessing
import time
from numpy import array
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader
torch.manual_seed(1)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
from sklearn.preprocessing import MinMaxScaler
#rotate a list by given offset, (negative offset, right to left) 
def rotateList(items, offset):
    return np.roll(items,offset)

def secondToMillisecond(val):
    return val*1000

def gaussiantimeseries(limit = 60, interval = 2):
    data = {}
    for t in range(0,limit,interval):
        data[t]=random.normal(loc = random.uniform(1,100), scale =1.5, size =1000).astype(float)
    return data

def createsignal(limit = 60, interval = 2):
    signal1, signal2, offset = [],[],[]
    limit = secondToMillisecond(limit)
    offset = random.randint(-10,10)
    data = gaussiantimeseries(limit, interval)
    for t in data:
        dist = data[t]
        signal1.append(random.choice(dist))
        signal2.append(random.choice(dist))
    return array(signal1).reshape((len(signal1), 1)), array(signal2).reshape((len(signal2), 1)), array(rotateList(signal2,offset)).reshape((len(signal2), 1)), offset

def createsignals(num = 50, limit = 60, interval = 2, normalize = False):
    input_seq = []
    data = []
    for i in range(num):
        sig1, sig2original, sig2, offset = createsignal(limit = limit)
        data.append([sig1, sig2original, sig2, offset])
        train_data = np.hstack((sig1,sig2)).astype(float)
        #train data without normalization
        if not normalize:
            train_data = torch.FloatTensor(train_data)
            input_seq.append((train_data, torch.FloatTensor([offset])))
        else:
            scaler = MinMaxScaler(feature_range=(-1, 1))
            train_data_normalized = scaler.fit_transform(train_data)
            train_data_normalized = torch.FloatTensor(train_data_normalized)
            input_seq.append((train_data_normalized, torch.FloatTensor([offset])))
    return input_seq, data

def createsignalsfortensor(num = 50, limit = 60, interval = 2, normalize = False):
    input_seq = []
    labels = []
    for i in range(num):
        sig1, sig2original, sig2, offset = createsignal(limit = limit)
        train_data = np.hstack((sig1,sig2)).astype(float)
        #train data without normalization
        if not normalize:
            train_data = torch.FloatTensor(train_data)
            input_seq.append(train_data)
            labels.append(torch.FloatTensor([offset]))
        else:
            scaler = MinMaxScaler(feature_range=(-1, 1))
            train_data_normalized = scaler.fit_transform(train_data)
            train_data_normalized = torch.FloatTensor(train_data_normalized)
            input_seq.append(train_data_normalized)
            labels.append(torch.FloatTensor([offset]))
    return input_seq, labels
# print(createsignal())

In [4]:
train_input, train_label = createsignalsfortensor(num=1000, limit = 60, interval = 1, normalize = False)
test_input, test_label = createsignalsfortensor()

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
train_label[0].float().shape

torch.Size([1])

In [12]:
class CustomDataset(torch.utils.data.Dataset):
  def __init__(self, data, labels):
        self.labels = labels
        self.data = data

  def __len__(self):
        return len(self.data)

  def __getitem__(self, index):
        X = self.data[index]
        y = self.labels[index]

        return X, y

In [13]:
train_data = CustomDataset(train_input,train_label)
# val_data = CustomDataset(test_input,test_label)
test_data = CustomDataset(test_input,test_label)

batch_size = 10

train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
# val_loader = DataLoader(val_data, shuffle=True, batch_size = batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

In [14]:
class LSTM(nn.Module):
    def __init__(self, input_size=2, output_size = 1, hidden_dim = 512, n_layers = 2, drop_prob=0):
        super(LSTM, self).__init__()
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        

        self.lstm = nn.LSTM(input_size, hidden_dim, n_layers, dropout=drop_prob, batch_first=True)
        self.dropout = nn.Dropout(drop_prob)
        self.fc = nn.Linear(hidden_dim, output_size)
        # self.sigmoid = nn.Sigmoid()
        
    def forward(self, x, hidden):
        batch_size = x.size(0)
        # x = x.long()
        lstm_out, hidden = self.lstm(x, hidden)
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        # lstm_out = self.dropout(lstm_out)
        out = self.fc(lstm_out)
        # out = self.sigmoid(out)
        
        out = out.view(batch_size, -1)
        out = out[:,-1]
        # print(out.shape)
        return out, hidden
    
    def init_hidden(self, batch_size):
        # print(batch_size)
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device))
        return hidden

In [15]:
model = LSTM()
model.to(device)

lr = 0.0001
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

In [16]:
epochs = 10
counter = 0
print_every = 100
clip = 5
valid_loss_min = np.Inf

model.train()
for i in range(epochs):
    # print(batch_size)
    print("Epoch {}/{}".format(i+1, epochs))
    print('-' * 20)
    epoch_loss = 0.0
    h = model.init_hidden(batch_size)
    model.zero_grad()
    for inputs, labels in train_loader:
        # print(inputs.shape)
        counter += 1
        h = tuple([e.data for e in h])
        inputs, labels = inputs.to(device), labels.to(device)

        output, h = model(inputs, h)
        # print("a",output.shape)
        # print("b",labels.squeeze().shape)
        optimizer.zero_grad()
        loss = criterion(output, labels.squeeze())
        # print(output)
        # print(labels)
        # break
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        epoch_loss += output.shape[0] * loss.item()
        if counter%print_every == 0:
        #     # print(batch_size)
        #     val_h = model.init_hidden(batch_size)
        #     val_losses = []
        #     model.eval()
        #     for inp, lab in val_loader:
        #         val_h = tuple([each.data for each in val_h])
        #         inp, lab = inp.to(device), lab.to(device)
        #         out, val_h = model(inp, val_h)
        #         val_loss = criterion(out, lab.squeeze())
        #         val_losses.append(val_loss.item())
                
            # model.train()
            print("Epoch: {}/{}".format(i+1, epochs),
                  "Step: {}".format(counter),
                  "Loss: {:.6f}".format(loss.item()))
            #       "Val Loss: {:.6f}".format(np.mean(val_losses)))
            # if np.mean(val_losses) <= valid_loss_min:
            #     # torch.save(model.state_dict(), './state_dict.pt')
            #     print('Validation loss decreased ({:.6f} --> {:.6f}).'.format(valid_loss_min,np.mean(val_losses)))
            #     valid_loss_min = np.mean(val_losses)
    # break
    print("Epoch {} Loss {}".format(i+1, epoch_loss / len(train_input)))

Epoch 1/10
--------------------
Epoch: 1/10 Step: 100 Loss: 32.085953
Epoch 1 Loss 30.959558176994324
Epoch 2/10
--------------------
Epoch: 2/10 Step: 200 Loss: 23.286352
Epoch 2 Loss 29.79758045196533
Epoch 3/10
--------------------
Epoch: 3/10 Step: 300 Loss: 26.004011
Epoch 3 Loss 30.50064385890961
Epoch 4/10
--------------------
Epoch: 4/10 Step: 400 Loss: 12.052190
Epoch 4 Loss 26.51044460296631
Epoch 5/10
--------------------
Epoch: 5/10 Step: 500 Loss: 9.414104
Epoch 5 Loss 21.610776422023772
Epoch 6/10
--------------------
Epoch: 6/10 Step: 600 Loss: 9.048156
Epoch 6 Loss 17.328953034877777
Epoch 7/10
--------------------
Epoch: 7/10 Step: 700 Loss: 7.753678
Epoch 7 Loss 11.322938318252563
Epoch 8/10
--------------------
Epoch: 8/10 Step: 800 Loss: 7.070020
Epoch 8 Loss 15.853488286733628
Epoch 9/10
--------------------
Epoch: 9/10 Step: 900 Loss: 19.618713
Epoch 9 Loss 15.815981690883637
Epoch 10/10
--------------------
Epoch: 10/10 Step: 1000 Loss: 18.262135
Epoch 10 Loss 11

In [17]:
test_losses = []
num_correct = 0
h = model.init_hidden(batch_size)

model.eval()
for inputs, labels in test_loader:
    h = tuple([each.data for each in h])
    inputs, labels = inputs.to(device), labels.to(device)
    # print(inputs)
    output, h = model(inputs, h)
    print(f"Offset {labels.squeeze().data}, Prediction {output.data}")
    test_loss = criterion(output, labels.squeeze())
    test_losses.append(test_loss.item())
    pred = torch.round(output)  # Rounds the output to 0/1
    correct_tensor = pred.eq(labels.squeeze().view_as(pred))
    correct = np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)

print("Test loss: {:.3f}".format(np.mean(test_losses)))
test_acc = num_correct/len(test_loader.dataset)
print("Test accuracy: {:.3f}%".format(test_acc*100))

Offset tensor([ 5.,  4.,  9., -2., -3., -6.,  1.,  6., -6.,  3.], device='cuda:0'), Prediction tensor([ 4.3864,  5.3206,  5.6908, -4.9569, -4.9633, -5.2299, -5.0490,  5.5100,
        -5.3521, -4.9862], device='cuda:0')
Offset tensor([-5., -8.,  2., -3., -3.,  3., -7., -2., -5., -9.], device='cuda:0'), Prediction tensor([-4.9775, -5.0612,  1.0339, -4.8749, -4.8601, -5.1333, -5.3361, -4.7561,
        -5.1246, -5.1495], device='cuda:0')
Offset tensor([ 4.,  3., -6., -9., -4.,  6., -7.,  3.,  3.,  6.], device='cuda:0'), Prediction tensor([ 5.3132,  4.7652, -5.0822, -5.2572, -5.2212,  5.5428, -5.2080, -5.0136,
         5.5068,  6.0613], device='cuda:0')
Offset tensor([ 7., -1.,  2., -7.,  1.,  8.,  4., -8., -1.,  7.], device='cuda:0'), Prediction tensor([ 5.6267, -0.5515, -5.0087, -5.2725, -4.1911,  6.2118,  5.8392, -5.3134,
         0.0498,  5.3421], device='cuda:0')
Offset tensor([  8.,   6., -10.,   9.,   5.,  -9.,   9.,  -5.,  -9.,   9.],
       device='cuda:0'), Prediction tensor([ 5.7